# Sélection de variables

La sélection de variables consiste à chercher un sous-ensemble de variables parmi un ensemble de variables. Les variables sélectionnées ne se sont pas tranformés et gardent ainsi leur signification initiale. La sélection des variables permet de trouver un modèle parcimonieux. Cependant il souvent difficile de trouver un sous-ensemble de variables stable même avec une validation croisée.

**Critères de sélection**

Il existe plusieurs critères permettant de mesurer la qualité d'une prédiction :
* Le critère d’information d'Akaïke (AIC) : retient les variables pertinentes lors de prévisions
* Le critère d’information bayésien (BIC) : vise la sélection de variables statistiquement significatives
dans le modèle.   
* Les métriques d'évaluation de modèles (MSE, MAE, accuracy, ...)
* ...

Ces critères sont équivalents aux R2, lorsqu'on compare des modèles avec le même nombre de variables.

**Remarque** :   
Dans le cadre d'une sélection de variables, le R2 ne permet pas d'obtenir le modèle optimale car il préviligie le modèle complexe (celui avec plus de variables).

L'objectif est de montrer comment mettre en oeuvre les techniques de sélection de variables avec Python. Il ne s'agira pas comparer les résultats des différentes méthodes de sélection

In [1]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from math import sqrt, log

from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

data = pd.read_csv('data/depSeuil.txt', sep=",")
data.columns = data.columns.str.upper()
data["STATION"] = pd.Categorical(data["STATION"], ordered=False)
data["JOUR"] = pd.Categorical(data["JOUR"], ordered=False)
data["O3OBS"] = pd.DataFrame(data["O3OBS"], dtype=np.float64)

data["SRMH2O"] = data["RMH2O"].map(lambda x: sqrt(x))
data["LNO2"] = data["NO2"].map(lambda x: log(x))
data["LNO"] = data["NO"].map(lambda x: log(x))

data = data.drop(["RMH2O", "NO2", "NO"], axis=1)
numerical_features = ["MOCAGE", "TEMPE", "VENTMOD", "VENTANG", "SRMH2O", "LNO2", "LNO"]
categorical_features = ["JOUR", "STATION"]

data = pd.get_dummies(data, columns=categorical_features, drop_first=True)

target_name = 'O3OBS'
train, test = train_test_split(data, test_size=0.2, random_state=125)
X_train = train.drop(target_name, axis=1)
Y_train = train[target_name]
X_test = test.drop(target_name, axis=1)
Y_test = test[target_name]

data.dtypes

O3OBS          float64
MOCAGE         float64
TEMPE          float64
VENTMOD        float64
VENTANG        float64
SRMH2O         float64
LNO2           float64
LNO            float64
JOUR_1           uint8
STATION_Als      uint8
STATION_Cad      uint8
STATION_Pla      uint8
STATION_Ram      uint8
dtype: object

# 1. Sélection de variable par pénalisation

Une régularisation **Lasso** type `L1` peut-être utilisée pour sélectionner des variables. On peut utiliser la version randomisée pour éviter l’instabilité des résultats. Les variables sélectionnées ont un coefficient non nul.

`Scikit-Learn` fournit un estimateur permet de récupérer les variables sélectionnées. Elles pourront être utilisées par un autre méthode. 

In [2]:
%%time
from sklearn.linear_model import Lasso

print('Nombre de variables intiales : {}'.format(X_train.shape[1]))

pipe = Pipeline([('feature_selection', SelectFromModel(Lasso(random_state=125))),
                 ('regressor', RandomForestRegressor(random_state=125))])

param_grid = {
              "regressor__max_features": [0.6, 0.8],
              "regressor__n_estimators": [50, 100, 200, 300],
              "regressor__min_samples_split": [2, 3, 4],
              "regressor__min_samples_leaf": [2, 3, 4]
             }

model = GridSearchCV(pipe, param_grid, cv=3, n_jobs=3)
_ = model.fit(X_train, Y_train)

print('Nombre de variables sélectionnées par le Lasso : {}'.format(model.best_estimator_[-1].n_features_))

predictions = model.predict(X_test)
mse = mean_squared_error(Y_test, predictions)
print("MSE = {0:.2f}".format(mse))

Nombre de variables intiales : 12
Nombre de variables sélectionnées par le Lasso : 6
MSE = 591.12
Wall time: 23.2 s


La régression **Ridge** conserve toutes les variables mais empêche les coefficients de prendre de trop grandes valeurs et limite ainsi la variance des prévision

La méthode **Elastic Net** permet de combiner la régression **ridge** et la régression **Lasso**, en introduisant les deux types de pénalités simultanément.

# 2. Recherche du meilleur modèle par sélection de variables

## 2.1 Recherche exhaustive

La recherche exhaustive consiste à tester toutes les combinaisons de variables pour sélectionner le meilleur modèle.

Cette solution est à éviter lorsqu'on a beaucoup de données car il y a 2<sup>p</sup> - 1 modèles à évaluer (`p` = nombre de variables)

## 2.2 La recherche pas à pas

## 2.2.1 Méthode descendante : back elimination

* On part du modèle complet avec toutes variables explicatives
* A chaque étape, on envèle la variable conduit au critère de sélection le plus faible
* La procédure s’arrête lorsque le critère ne décroît plus.

In [3]:
%%time
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import Lasso

lasso = Lasso(random_state=125)

pipe = Pipeline([('feature_selection', SequentialFeatureSelector(lasso, 
                                                                 n_features_to_select=6, 
                                                                 direction='backward')),
                 ('regressor', RandomForestRegressor(random_state=125))])

param_grid = {
              "regressor__max_features": [0.6, 0.8],
              "regressor__n_estimators": [50, 100, 200, 300],
              "regressor__min_samples_split": [2, 3, 4],
              "regressor__min_samples_leaf": [2, 3, 4]
             }

model = GridSearchCV(pipe, param_grid, cv=3, n_jobs=3)
_ = model.fit(X_train, Y_train)

predictions = model.predict(X_test)
mse = mean_squared_error(Y_test, predictions)
print("MSE = {0:.2f}".format(mse))

MSE = 592.14
Wall time: 1min 6s


## 2.2.2  Méthode ascendante : foward selection

* À chaque étape, une variable est ajoutée au modèle. Il s'agit la variable qui permet de réduire au mieux le critère de sélection 

* La procédure s’arrête lorsque toutes les variables sont introduites ou lorsque le critère de sélection ne décroît plus.

In [4]:
%%time
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import Lasso

lasso = Lasso(random_state=125)

pipe = Pipeline([('feature_selection', SequentialFeatureSelector(lasso, 
                                                                 n_features_to_select=6, 
                                                                 direction='forward')),
                 ('regressor', RandomForestRegressor(random_state=125))])

param_grid = {
              "regressor__max_features": [0.6, 0.8],
              "regressor__n_estimators": [50, 100, 200, 300],
              "regressor__min_samples_split": [2, 3, 4],
              "regressor__min_samples_leaf": [2, 3, 4]
             }

model = GridSearchCV(pipe, param_grid, cv=3, n_jobs=3)
_ = model.fit(X_train, Y_train)

predictions = model.predict(X_test)
mse = mean_squared_error(Y_test, predictions)
print("MSE = {0:.2f}".format(mse))

MSE = 592.14
Wall time: 1min 10s


## 2.1.3  Stepwise selection

C'est un mixte de forward selection et backward elimination. Il introduit une étape d'élimination de variable après chaque étape de sélection afin de retirer du modèle d'éventuels variables qui seraient devenues moins indispensables du fait de la présence de celles nouvellement introduite.

# 3. D'autres techniques de sélection de variables 

## 3.1 Recursive feature elimination (RFE)

La méthode RFE fonctionne par suppression récursive des features en se basant sur l'importance des variables :
* Le nombre de variables à sélectionner est fixé au départ.
* On part sur le modèle complet (avec toutes les variables explicatives)
* A chaque étape, on  élimine la variable la moins importante
* Jusqu'à atteindre le nombre de variables à sélectionner

In [5]:
%%time
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

pipe = Pipeline([('feature_selection', RFE(SVR(kernel="linear"), n_features_to_select=6, step=1)),
                 ('regressor', RandomForestRegressor(random_state=125))])

param_grid = {
              "regressor__max_features": [0.6, 0.8],
              "regressor__n_estimators": [50, 100, 200, 300],
              "regressor__min_samples_split": [2, 3, 4],
              "regressor__min_samples_leaf": [2, 3, 4]
             }

model = GridSearchCV(pipe, param_grid, cv=3, n_jobs=3)
_ = model.fit(X_train, Y_train)

predictions = model.predict(X_test)
mse = mean_squared_error(Y_test, predictions)
print("MSE = {0:.2f}".format(mse))

MSE = 696.60
Wall time: 1min 10s


## 3.2 Recursive feature elimination with cross-validation (RFECV)

Comme son nom l'indique, le RFECV ajoute une cross-validation au RFE

In [6]:
%%time
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR

pipe = Pipeline([('feature_selection', RFECV(SVR(kernel="linear"),
                                             min_features_to_select=6,
                                             step=1,
                                             cv=3)),
                 ('regressor', RandomForestRegressor(random_state=125))])

param_grid = {
              "regressor__max_features": [0.6, 0.8],
              "regressor__n_estimators": [50, 100, 200, 300],
              "regressor__min_samples_split": [2, 3, 4],
              "regressor__min_samples_leaf": [2, 3, 4]
             }

model = GridSearchCV(pipe, param_grid, cv=3, n_jobs=3)
_ = model.fit(X_train, Y_train)

predictions = model.predict(X_test)
mse = mean_squared_error(Y_test, predictions)
print("MSE = {0:.2f}".format(mse))

MSE = 588.90
Wall time: 2min 18s


## 3.3 Suppression des variables à faible variance

`Scikit-Learn` fournit un estimateur permet de filter les variables avec une faible variance.

In [7]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVR

pipe = Pipeline([('feature_selection', VarianceThreshold(threshold=0.1)),
                 ('regressor', RandomForestRegressor(random_state=125))])

param_grid = {
              "regressor__max_features": [0.6, 0.8],
              "regressor__n_estimators": [50, 100, 200, 300],
              "regressor__min_samples_split": [2, 3, 4],
              "regressor__min_samples_leaf": [2, 3, 4]
             }

model = GridSearchCV(pipe, param_grid, cv=3, n_jobs=3)
_ = model.fit(X_train, Y_train)

predictions = model.predict(X_test)
mse = mean_squared_error(Y_test, predictions)
print("MSE = {0:.2f}".format(mse))

MSE = 586.23


## 3.4 Sélection des variables basée sur un test statistique

`SelectKBest` permet de sélectionner les `k` meilleurs prédicteurs à l'aide d'une fonction score :
* Classification : chi2, f_classif, mutual_info_classif
* Regression: f_regression, mutual_info_regression

In [8]:
from sklearn.feature_selection import SelectKBest, f_regression

pipe = Pipeline([('feature_selection',  SelectKBest(f_regression, k=6)),
                 ('regressor', RandomForestRegressor(random_state=125))])

param_grid = {
              "regressor__max_features": [0.6, 0.8],
              "regressor__n_estimators": [50, 100, 200, 300],
              "regressor__min_samples_split": [2, 3, 4],
              "regressor__min_samples_leaf": [2, 3, 4]
             }

model = GridSearchCV(pipe, param_grid, cv=3, n_jobs=3)
_ = model.fit(X_train, Y_train)

predictions = model.predict(X_test)
mse = mean_squared_error(Y_test, predictions)
print("MSE = {0:.2f}".format(mse))

MSE = 658.63


**Sources :**    
[Sélection de modèle en régression linéaire, Philippe Besse](https://www.math.univ-toulouse.fr/~besse/Wikistat/pdf/st-m-app-linSelect.pdf)  
[Scikit-learn Documentation](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection)